# Build Chat Bot
This chatbot that built only use the language model. It will related concepts:
- Conversational RAG: enable chat bot with external source data
- Agents: chat bot can take actions
Concept:
- Chat Model: chatbot interface is based around message rather than raw text, Chat Model is best suited.
- Prompt Templates
- Chat History
- LangSmith to debugging

In [3]:
from langchain_community.llms import Ollama
model = Ollama(model="llama3")

In [4]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

In [5]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi! I'm Bob")])
# so you can re-check the logs in langsmith

"AI: Hi Bob! It's nice to meet you. Is there something I can help you with or would you like to chat?"

In [6]:
from langchain_core.messages import AIMessage

model.invoke(
  [
  HumanMessage(content="Hi! I'm Bob"),
  AIMessage(content="Hello Bob! How can I assist you today?"),
  HumanMessage(content="What's my name?"),
  ]
)

'Bob! According to our conversation, your name is indeed "Bob"!'

In [8]:
# 1. message history
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}
def get_session_history(session_id: str) -> BaseChatMessageHistory:
  if session_id not in store:
    store[session_id] = ChatMessageHistory()
  return store[session_id]

with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [11]:
# but now you need to create a new session id
config = {"configurable": {"session_id": "abc2"}}

response = with_message_history.invoke(
  [HumanMessage(content="Hi! I'm Bob")],
  config=config,
)
print(response)
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)
print(response)

Error in RootListenersTracer.on_llm_end callback: KeyError('message')


"AI: Hi Bob! It's nice to meet you. Is there something I can help you with or would you like to chat?"

In [13]:
# 2. Prompt Template
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

response = chain.invoke({"messages": [HumanMessage(content="hi! I'm bob")]})
response

"Hi Bob! It's great to meet you! How can I assist you today? Do you have any specific questions or topics you'd like to discuss? I'm here to help with anything from answering general knowledge questions, providing information on a particular topic, offering suggestions, or just chatting. Let me know what's on your mind, and I'll do my best to be helpful!"

In [14]:
# can now wrap this in the same Messages History object as before
with_message_history = RunnableWithMessageHistory(chain, get_session_history)
config = {"configurable": {"session_id": "abc5"}}
response = with_message_history.invoke(
  [HumanMessage(content="Hi! I'm Jim")],
  config=config,
)
print(response)
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)
print(response)

Hi Jim! Nice to meet you. I'm here to help with any questions or tasks you may have. What's on your mind today? Do you need assistance with something specific, or just looking for some general information or ideas? Let me know and I'll do my best to assist you!
Jim! You've introduced yourself as Jim, so that's correct! If you ever forget or want to double-check, feel free to ask me anytime. Now, what else can I help you with today? Do you have a specific question or topic in mind, or would you like some suggestions or ideas?


In [15]:
# again, with the prompt template and the model
prompt = ChatPromptTemplate.from_messages(
  [
    (
      "system",
      "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
    ),
    MessagesPlaceholder(variable_name="messages"),
  ]
)

chain = prompt | model

# response = chain.invoke(
#   {"messages": [HumanMessage(content="hi! I'm bob")], "language": "Spanish"}
# )

with_message_history = RunnableWithMessageHistory(
  chain,
  get_session_history,
  input_messages_key="messages",
)
config = {"configurable": {"session_id": "abc11"}}
response = with_message_history.invoke(
  {"messages": [HumanMessage(content="hi! I'm todd")], "language": "Spanish"},
  config=config,
)
print(response)
response = with_message_history.invoke(
  {"messages": [HumanMessage(content="whats my name?")], "language": "Spanish"},
  config=config,
)
print(response)

Hola Todd! Soy tu asistente amable y estoy aquí para ayudarte en lo que necesites. ¿En qué puedo ayudarte hoy?
¡Hola Todd! ¡Eso es fácil! Tu nombre es Todd, ¿correcto?


In [18]:
# 3. Manage conversation history
# Painpoint: If unmanaged, the conversation history can grow indefinitely, consuming memory and sizing.
from langchain_core.runnables import RunnablePassthrough

# example to filter with the last 10 messages
def filter_messages(messages, k=10):
  return messages[-k:]

chain = (
  RunnablePassthrough.assign(messages=lambda x: filter_messages(x["messages"]))
  | prompt
  | model
)
messages = [
  HumanMessage(content="hi! I'm bob"),
  AIMessage(content="hi!"),
  HumanMessage(content="I like vanilla ice cream"),
  AIMessage(content="nice"),
  HumanMessage(content="whats 2 + 2"),
  AIMessage(content="4"),
  HumanMessage(content="thanks"),
  AIMessage(content="no problem!"),
  HumanMessage(content="having fun?"),
  AIMessage(content="yes!"),
]

In [19]:
# Let's now try it out! If we create a list of messages more than 10 messages long, we can see what it no longer remembers information in the early messages.
response = chain.invoke(
  {
    "messages": messages + [HumanMessage(content="what's my name?")],
    "language": "English",
  }
)
response

"I'm happy to help!\n\nUnfortunately, I don't have any information about your name. You didn't introduce yourself or provide any identifying information. If you'd like, we can start fresh and explore more topics together! What would you like to talk about or ask next?"

In [21]:
# but if we ask about information that is within the last ten messages, it still remembers it
response = chain.invoke(
  {
    "messages": messages + [HumanMessage(content="what's my fav ice cream")],
    "language": "English",
  }
)
response

'You already told me that your favorite ice cream is vanilla!'

In [22]:
# wrap this in the Message History
with_message_history = RunnableWithMessageHistory(
  chain,
  get_session_history,
  input_messages_key="messages",
)

config = {"configurable": {"session_id": "abc20"}}
response = with_message_history.invoke(
  {
    "messages": [HumanMessage(content="whats my favorite ice cream?")],
    "language": "English",
  },
  config=config,
)
response

In [25]:
# Implement with streaming
config = {"configurable": {"session_id": "abc15"}}
for r in with_message_history.stream(
  {
    "messages": [HumanMessage(content="hi! I'm todd. tell me a joke")],
    "language": "English",
  },
  config=config,
):
  print(r, end="|")

Nice| to| meet| you|,| Todd|!

|Here|'s| one|:

|Why| don|'t| scientists| trust| atoms|?

|Because| they| make| up| everything|!

|Hope| that| made| you| gig|gle|!| Do| you| want| another| one|?||

In [26]:
for r in with_message_history.stream(
  {
    "messages": [HumanMessage(content="What is my name?, and tell me joke that you said before")],
    "language": "English",
  },
  config=config,
):
  print(r, end="|")

Nice| to| meet| you| too|,| Todd|!

|As| for| your| question|,| I|'d| be| happy| to| remind| you| -| your| name| is| Todd|!

|And|,| would| you| like| to| hear| the| joke| again|?
|Why| don|'t| scientists| trust| atoms|?| Because| they| make| up| everything|!| Hope| that| made| you| smile| again|!||